## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [1]:
%pip install requests requests_cache bs4

import requests, requests_cache, sqlite3, random, json, itertools, datetime
from bs4 import BeautifulSoup

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

Note: you may need to restart the kernel to use updated packages.


In [7]:
class Item:

    def __init__(self, url, id, name, state, price):
        self.title = name
        self.id = id
        self.url = url
        self.price = Decimal(price.replace(',', '.').strip())
        self.stock_state = state
        self.last_update = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    def update_time():
        self.last_update = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")


class Parser:

    def __init__(self, soup):
        self.urls = []
        self.names = []
        self.ids = []
        self.prices = []
        self.stock_states = []
        self.soup = soup


    def parse_urls(self):
        for tag in self.soup.find_all('div'):
            a = tag.find('a')
            if a is not None:
                if a.has_attr('href'):
                    url = tag.find('a')['href']
                    if url not in self.urls and url.startswith('/'):
                        self.urls.append(url)
    
    def parse_data(self, searched_attr):
        list = []
        for tag in self.soup.find_all('div'):
            a = tag.find('a')
            if a is not None:
                if a.has_attr(searched_attr):
                    id = tag.find('a')[searched_attr]
                    if id not in list:
                        list.append(id)
        return list


    def parse(self):
        id_attr = 'data-impression-id'
        name_attr = 'data-impression-name'
        state_attr = 'data-impression-dimension13'
        price_attr = 'data-impression-metric2'

        self.parse_urls()
        self.ids = self.parse_data(id_attr)
        self.names = self.parse_data(name_attr)
        self.stock_states = self.parse_data(state_attr)
        self.prices = self.parse_data(price_attr)

    
    def get_item_list(self):
        items = []
        for (url, id, name, state, price) in zip(self.urls, self.ids, self.names, self.stock_states, self.prices): 
            item = Item(url, id, name, state, price)
            items.append(item)
        return items


### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [8]:
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"
def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()
        
        #zde dopiste kod, predpokladana delka cca 50 radku
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'cs-CZ',
            'Prefer': 'safe',
            'Referer': 'https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm',
            'Content-Type': 'application/json; charset=utf-8',
            'cache-control': 'no-cache',
            'X-Requested-With': 'XMLHttpRequest',
            'Request-Id': '|088e716b77424a70b1c7266f5224f7b6.0011803675cb4c9b',
            'Origin': 'https://www.alza.cz',
            'Connection': 'keep-alive',
            'TE': 'Trailers',
        }

        data = '{"idCategory":18863907,"producers":"","parameters":[],"idPrefix":0,"prefixType":0,"page":1,"pageTo":4,"inStock":false,"newsOnly":false,"commodityStatusType":null,"upperDescriptionStatus":0,"branchId":-2,"sort":0,"categoryType":1,"searchTerm":"","sendProducers":false,"layout":0,"append":false,"leasingCatId":null,"yearFrom":null,"yearTo":null,"artistId":null,"minPrice":-1,"maxPrice":-1,"shouldDisplayVirtooal":false,"callFromParametrizationDialog":false,"commodityWearType":null,"scroll":5712,"hash":"#f&cst=null&cud=0&pg=1-2&prod=","counter":1}'
        
        response = s.post('https://www.alza.cz/Services/EShopService.svc/Filter', headers=headers, data=data)

        soup = BeautifulSoup( json.loads(response.content)['d']['Boxes'],'html.parser')

        parser = Parser(soup)
        parser.parse()
        items = parser.get_item_list()


        for product in items:
            c.execute("SELECT EXISTS(SELECT 1 FROM products WHERE id=? AND url=?  LIMIT 1)", (product.id, product.url))
            new_record = c.fetchone()
            if new_record[0] == 1:
                c.execute("SELECT EXISTS(SELECT 1 FROM products WHERE ((title!=?) OR (price!=?) OR (stock_state!=?)) AND (id=? AND url=?)  LIMIT 1)", (product.title, product.price, product.stock_state, product.id, product.url))
                record = c.fetchone()
                if record[0] == 1:
                    c.execute("SELECT id, url, title, price, stock_state, last_update FROM products WHERE id=? AND url=?", (product.id, product.url))
                    actual_record = c.fetchone()
                    c.execute("INSERT into products_history values (?, ?, ?, ?, ?, ?)", (actual_record[0], actual_record[1], actual_record[2], actual_record[3], actual_record[4], actual_record[5]))
                    c.execute("UPDATE products values price = ?, stock_state = ?, last_update = ?, ?, ?)", (product.price, product.stock_state, product.last_update))
            else:
                print("test")
                c.execute("INSERT into products values (?, ?, ?, ?, ?, ?)", (product.id, product.url, product.title, product.price, product.stock_state, product.last_update))


        conn.commit()        
        c.close()

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [9]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])

print("OK")        

InvalidOperation: [<class 'decimal.ConversionSyntax'>]

### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

In [ ]:
fuu